In [1]:
import multiprocessing
multiprocessing.set_start_method("fork")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm 

feat_ml_df = pd.read_pickle("data/deleteme.pkl")#.sort_values("Date")
gender_df = pd.read_csv("data/fighter_genders.csv")
feat_ml_df2 = feat_ml_df.merge(gender_df, on="FighterID", how="left")
feat_ml_df2 = feat_ml_df2.sort_values("Date")
# feat_ml_df2["gender"].isnull().any()
feat_ml_df.head()

,FighterID,OpponentID,oldFighterEloAD,oldOpponentEloAD,predTargetAD,targetAD,Date,newFighterEloAD,newOpponentEloAD,oldEloDiffAD,...,FighterOpen,OpponentOpen,FighterCloseLeft,FighterCloseRight,OpponentCloseLeft,OpponentCloseRight,p_fighter,p_opponent,p_fighter_midpoint,p_fighter_implied
0,2335629/bj-penn,2335885/jens-pulver,0.842355,0.364097,0.478257,2.645751,2007-06-23,1.275853,-0.069401,0.478257,...,-490.0,340.0,-357.0,-330.0,225.0,280.0,0.830508,0.227273,0.801618,0.785142
1,2335676/joe-lauzon,2354360/brandon-melendez,0.091024,0.000000,0.091024,2.000000,2007-06-23,0.472819,-0.381795,0.091024,...,-430.0,330.0,-700.0,-475.0,355.0,425.0,0.811321,0.232558,0.789381,0.777217
2,2335447/anderson-silva,2335475/nate-marquardt,0.227196,0.736307,-0.509111,-1.000000,2007-07-07,0.129018,0.834485,-0.509111,...,-170.0,150.0,-145.0,-145.0,125.0,125.0,0.629630,0.400000,0.614815,0.611511
3,2335302/heath-herring,2335521/antonio-rodrigo-nogueira,0.106410,0.000000,0.106410,-2.645751,2007-07-07,-0.444022,0.550432,0.106410,...,470.0,-810.0,500.0,500.0,-700.0,-700.0,0.175439,0.890110,0.142664,0.164646
4,2335694/frankie-edgar,2335717/mark-bocek,0.000000,0.000000,0.000000,1.414214,2007-07-07,0.282843,-0.282843,0.000000,...,-260.0,180.0,-230.0,-230.0,190.0,190.0,0.722222,0.357143,0.682540,0.669118


In [3]:
feat_cols = [
    "oldEloDiffordinal_fighter_result", 
    "oldEloDiffsubmission_fighter_result",
    "oldEloDifftko_ko_fighter_result",
    "oldEloDiffdecision_fighter_result",
#     "oldEloDifffinish_fighter_result",
#     "oldEloDifffighter_result_time_left",
    
    'oldEloDiffAD', 
    'oldEloDiffADTB', 
    'oldEloDiffADTM', 
    'oldEloDiffKD',
    'oldEloDiffRV', 
    'oldEloDiffSCBL', 
    'oldEloDiffSCHL', 
    'oldEloDiffSGBL',
    'oldEloDiffSGHL', 
    'oldEloDiffSM', 'oldEloDiffSSL', 'oldEloDiffTDL',
    'oldEloDiffTDS', 
    'oldEloDiffTD_fails', 
    'oldEloDiffTSL',
    'oldEloDiffclinch_strikes_landed', 
    'oldEloDiffdistance_strikes_landed',
    'oldEloDiffground_strikes_landed', 
    'oldEloDiffstanding_strikes',
    
    'oldEloDiffWin',
    
    "t_since_last_fight_log_diff", 
#     "t_since_last_fight_diff",
    "total_fights_sqrt_diff", 
    "total_ufc_fights_diff",
    
    "age_diff", "reach_diff", 
#     "weight_diff", 
    "log_weight_diff",
    "height_diff",
]

In [21]:
from model.mma_hier_log_reg_stan import HierPcaSymmetricModel

hier_model = HierPcaSymmetricModel(feat_cols, n_pca=8, 
                                   beta_prior_std=1.0, intra_group_std=0.1, 
                                   mcmc=False)

In [24]:
from model_selection.cross_val_pipeline import TimeSeriesCrossVal

temp_df = feat_ml_df2.dropna(subset=[*feat_cols, "targetWin"])
tscv = TimeSeriesCrossVal(n_splits=4)
fold_pred_df = tscv.get_cross_val_preds(hier_model, temp_df)
tscv.score_preds()

training on date range: 2007-06-23 2012-04-14
Initial log joint probability = -1950.14
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      13      -322.154   0.000124594     0.0122169      0.6989      0.6989       14   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
training on date range: 2007-06-23 2015-04-04
Initial log joint probability = -3182.24
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      16      -613.242   9.01145e-05    0.00563112           1           1       18   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
training on date range: 2007-06-23 2017-11-18
Initial log joint probability = -3186.35
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      14       -979.58   0.000132726    0.00525005           1           1   

,test_fold,min_test_date,max_test_date,n_test_days,n_test_fights,log_loss,accuracy_score,ml_log_loss
0,1,2012-04-21,2015-04-04,108,478,0.624514,0.631799,0.613181
1,2,2015-04-11,2017-11-18,108,557,0.660574,0.588869,0.661171
2,3,2017-11-25,2020-02-15,108,750,0.640064,0.626667,0.642069
3,4,2020-02-22,2022-03-19,107,891,0.624973,0.635241,0.628963
